<a href="https://colab.research.google.com/github/alvin-98/mental-health-survey-2014/blob/main/Mental_Health_Survey_2014.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Analysis of Mental Health Survey - 2014

##### You are now on Google Colab, here you will be able to interact with each cell, run and tinker with the code, all on your browser.
**Note :**
* **You will be able to run the code only after signing into colab with your Google account.**
* **If the graphs do not show up or have loaded incorrectly, click `Runtime` in the tool bar and select `Run all`.**<br>
* **Since the notebook isn't authored by Google, a warning will show up citing the same.**<br>
* **Running the program is absolutely safe. Therefore, you can click `Run Anyway`.**


In [36]:
pip install plotly==5.3.1

In [37]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [38]:
df = pd.read_csv('https://raw.githubusercontent.com/alvin-98/mental-health-survey-2014/main/survey.csv')
df.shape

(1259, 27)

In [39]:
df.head()

,Timestamp,Age,Gender,Country,state,self_employed,family_history,treatment,work_interfere,no_employees,remote_work,tech_company,benefits,care_options,wellness_program,seek_help,anonymity,leave,mental_health_consequence,phys_health_consequence,coworkers,supervisor,mental_health_interview,phys_health_interview,mental_vs_physical,obs_consequence,comments
0,2014-08-27 11:29:31,37,Female,United States,IL,NaN,No,Yes,Often,6-25,No,Yes,Yes,Not sure,No,Yes,Yes,Somewhat easy,No,No,Some of them,Yes,No,Maybe,Yes,No,NaN
1,2014-08-27 11:29:37,44,M,United States,IN,NaN,No,No,Rarely,More than 1000,No,No,Don't know,No,Don't know,Don't know,Don't know,Don't know,Maybe,No,No,No,No,No,Don't know,No,NaN
2,2014-08-27 11:29:44,32,Male,Canada,NaN,NaN,No,No,Rarely,6-25,No,Yes,No,No,No,No,Don't know,Somewhat difficult,No,No,Yes,Yes,Yes,Yes,No,No,NaN
3,2014-08-27 11:29:46,31,Male,United Kingdom,NaN,NaN,Yes,Yes,Often,26-100,No,Yes,No,Yes,No,No,No,Somewhat difficult,Yes,Yes,Some of them,No,Maybe,Maybe,No,Yes,NaN
4,2014-08-27 11:30:22,31,Male,United States,TX,NaN,No,No,Never,100-500,Yes,Yes,Yes,No,Don't know,Don't know,Don't know,Don't know,No,No,Some of them,Yes,Yes,Yes,Don't know,No,NaN


* The above is a quick look at the dataset.
* The dataset contains 1259 rows are 27 columns. We're going to investigate to check and fix bad quality data.

###Data Cleaning

In [40]:
df[(df['Country'] != 'United States')]['state'].unique()

array([nan, 'NY', 'MD', 'IL', 'UT'], dtype=object)

In [41]:
df[(df['Country'] != 'United States') & (df['state'].isna() == False)]

,Timestamp,Age,Gender,Country,state,self_employed,family_history,treatment,work_interfere,no_employees,remote_work,tech_company,benefits,care_options,wellness_program,seek_help,anonymity,leave,mental_health_consequence,phys_health_consequence,coworkers,supervisor,mental_health_interview,phys_health_interview,mental_vs_physical,obs_consequence,comments
319,2014-08-27 14:28:28,29,male,Latvia,NY,No,No,No,NaN,26-100,No,Yes,No,Yes,No,No,Don't know,Somewhat easy,Maybe,Maybe,Some of them,Some of them,No,No,Don't know,No,NaN
488,2014-08-27 16:21:54,26,Male,Israel,MD,No,No,No,Sometimes,6-25,No,Yes,No,No,No,No,Yes,Very easy,Maybe,No,Some of them,Yes,No,Maybe,Yes,No,Israel has public health insurance for everyon...
989,2014-08-29 09:10:58,8,A little about you,"Bahamas, The",IL,Yes,Yes,Yes,Often,1-5,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Very easy,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,
1179,2014-09-26 10:05:55,25,female,Bulgaria,UT,No,No,Yes,Sometimes,26-100,Yes,Yes,No,No,No,No,Don't know,Don't know,Maybe,Maybe,Yes,Some of them,Maybe,No,No,No,NaN


In [42]:
df.loc[(df['Country'] != 'United States') & (df['state'].isna()==False), 'state'] = np.nan

* The 'state' column contains the response to the following question - *If you live in the United States, which state or territory do you live in?* 
* Few countries (which are not the US) have US states selected against them. The values of the 'state' column in such rows have been replaced with 'NaN'


In [43]:
index_names = df[(df['Age']<19)|(df['Age']>75)].index
df.drop(index_names, inplace=True)

* There are a total of 15 rows where age has very low or negative values or extremely high values. We will be dropping all such rows.

In [44]:
df['Gender'].unique()

array(['Female', 'M', 'Male', 'male', 'female', 'm', 'Male-ish', 'maile',
       'Trans-female', 'Cis Female', 'F', 'Cis Male', 'Woman', 'f', 'Mal',
       'Male (CIS)', 'queer/she/they', 'non-binary', 'Femake', 'woman',
       'Make', 'Nah', 'Enby', 'fluid', 'Genderqueer', 'Female ',
       'Androgyne', 'Agender', 'cis-female/femme', 'Guy (-ish) ^_^',
       'male leaning androgynous', 'Male ', 'Man', 'Trans woman', 'msle',
       'Neuter', 'Female (trans)', 'queer', 'Female (cis)', 'Mail',
       'cis male', 'Malr', 'femail', 'Cis Man',
       'ostensibly male, unsure what that really means'], dtype=object)

In [45]:
male_std = [male for male in df['Gender'].unique() if ('Male' in male) and ('f' not in male)] + ['m','M','maile','male', 'something kinda male?', 'Mal', 'Make', 'Guy (-ish) ^_^', 'Malr', 'Man', 'msle', 'Mail', 'cis male', 'Malr', 'Cis Man', 'ostensibly male, unsure what that really means']
df['Gender'].replace(to_replace=male_std, value='Male', inplace=True)
female_std = ['female', 'Cis Female', 'F', 'f', 'Woman', 'Femake', 'woman', 'Female ', 'cis-female/femme', 'Female (cis)', 'femail']
df['Gender'].replace(to_replace=female_std, value='Female', inplace=True)
others = [oth for oth in df['Gender'].unique() if oth!='Male' and oth!='Female']
df['Gender'].replace(to_replace=others, value='Others', inplace=True)
df['Gender'].value_counts()

Male      983
Female    246
Others     15
Name: Gender, dtype: int64

* Transforming various male/female responses to a standard form and thereby fixing errors/typos
* Grouping all other responses into 'Others' category as the number of responses is comparatively very low

In [46]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1244 entries, 0 to 1258
Data columns (total 27 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   Timestamp                  1244 non-null   object
 1   Age                        1244 non-null   int64 
 2   Gender                     1244 non-null   object
 3   Country                    1244 non-null   object
 4   state                      732 non-null    object
 5   self_employed              1226 non-null   object
 6   family_history             1244 non-null   object
 7   treatment                  1244 non-null   object
 8   work_interfere             984 non-null    object
 9   no_employees               1244 non-null   object
 10  remote_work                1244 non-null   object
 11  tech_company               1244 non-null   object
 12  benefits                   1244 non-null   object
 13  care_options               1244 non-null   object
 14  wellness

In [47]:
print(f"The number of 'Null' values by column: \n\n{df.isnull().sum()}")

The number of 'Null' values by column: 

Timestamp                       0
Age                             0
Gender                          0
Country                         0
state                         512
self_employed                  18
family_history                  0
treatment                       0
work_interfere                260
no_employees                    0
remote_work                     0
tech_company                    0
benefits                        0
care_options                    0
wellness_program                0
seek_help                       0
anonymity                       0
leave                           0
mental_health_consequence       0
phys_health_consequence         0
coworkers                       0
supervisor                      0
mental_health_interview         0
phys_health_interview           0
mental_vs_physical              0
obs_consequence                 0
comments                     1085
dtype: int64


**Note: In the subsequent visuals, 'n' refers to the number of respondents who have answered the corresponding question the chart addresses.**

### Respondent Profile

In [48]:
fig = go.Figure(data=go.Pie(values=df.Gender.value_counts().values, 
             labels=df.Gender.value_counts().index,
             name='',
             hovertemplate = "# of respondents responding - %{label}: %{value}"))

fig.update_traces(marker=dict(colors=['skyblue', 'pink', 'lightgray'], line=dict(color='#000000', width=0.6)))

fig.add_annotation(
            text="% of respondents",
            showarrow=False,
            xshift=5,
            yshift=-150,
            font={'size':11})

fig.update_layout(
                  height=500,
                  title_y=0.92,
                  margin=dict(t=170),
                  legend_x=0.8, 
                  legend_y=0.5, 
                  title={'text':'Respondent Profile (I) <br>(n = 1244)', 'x':0.51}, titlefont=dict(size=14))

fig.show()

* The 'Male' gender represents ~79% of the sample while the 'Female' gender represents ~20% of the respondents
* The rest of the sample is made up of non-binary, transgender and gender fluid people

In [49]:
# Country representation
fig = go.Figure(go.Bar(x=df.Country.value_counts().index, 
                       y=df.Country.value_counts().values, 
                       text=df.Country.value_counts().values, 
                       textposition='outside',
                       marker_color='#D4355A',
                       hoverinfo='skip',
                       ),
                )
fig.update_layout(plot_bgcolor="#FFFFFF", 
                  title={'text':'Respondent Profile (II) <br>(n = 1244)', 'x':0.5}, 
                  titlefont=dict(size=14),
                  xaxis_tickangle=-45, 
                  xaxis_title='Countries', 
                  yaxis_title='# of respondents',
                  uniformtext_minsize=6,
                  uniformtext_mode='hide')
fig.update_xaxes(showgrid=False, linecolor="gray", linewidth=0.5, titlefont=dict(size=12))
fig.update_yaxes(tickvals=np.arange(0,1000,100) , showgrid=False, linecolor="gray", linewidth=0.5, titlefont=dict(size=12))
#fig.layout.title.font.family='Calibri'

fig.show()

* United states and United kingdom represents ~75% of the data 

In [50]:
# Number of employees and age distribution
fig = make_subplots(rows=1, cols=2, subplot_titles=['Company Size', 'Age of Respondents'])

fig.add_trace(go.Bar(x=['1-5', '6-25', '26-100', '100-500', '500-1000', '> 1000'], 
                     y=df.no_employees.value_counts()[['1-5', '6-25', '26-100', '100-500', '500-1000', 'More than 1000']].values, 
                     text=df.no_employees.value_counts()[['1-5', '6-25', '26-100', '100-500', '500-1000', 'More than 1000']].values,
                     textposition='outside',
                     marker_color='#D4355A',
                     hoverinfo = 'skip'), 
              row=1, 
              col=1)
bins=[i*10 for i in range(1,9)]
counts, bins = np.histogram(df.Age, bins=bins)
fig.add_trace(go.Bar(x=['11-20', '21-30', '31-40', '41-50', '51-60', '61-70', '71-80'], 
                     y=counts, 
                     text=counts, 
                     textposition='outside',
                     marker_color='#D4355A',
                     hoverinfo = 'skip'), 
              row=1, 
              col=2)
fig.update_layout(bargap=0.5, 
                  xaxis_tickangle=-0, 
                  plot_bgcolor="#FFFFFF", 
                  uniformtext_minsize=5, 
                  uniformtext_mode='hide')
                  
fig.update_annotations(font_size=14)                  
fig.add_annotation(x=0,
                   y=0,
            text="Respondent Profile (III) <br>(n = 1244)",
            showarrow=False,
            xshift=520,
            yshift=400,            
            font={'size':14})

fig.update_traces(marker=dict(line=dict(color='#000000', width=0.6)))

fig.update(layout_showlegend=False)
fig.update_xaxes(showgrid=False, linecolor="gray", linewidth=0.5)
fig.update_yaxes(range=[0,800], showgrid=True, linecolor="gray", linewidth=0.5)
fig['layout']['xaxis']['title']='No. of employees'
fig['layout']['xaxis']['titlefont']=dict(size=12)
fig['layout']['xaxis2']['title']='Age groups'
fig['layout']['xaxis2']['titlefont']=dict(size=12)
fig['layout']['yaxis']['title']='# of respondents'
fig['layout']['yaxis']['titlefont']=dict(size=12)
fig['layout']['yaxis2']['title']='# of respondents'
fig['layout']['yaxis2']['titlefont']=dict(size=12)
fig.show()

* ~58% respondents come from small size companies with 1-100 employees
* ~22% respondents come from large companies with greater than 1000 employees

* ~84% of the respondents are within the 20-40 years age group

In [51]:
# Trying to show all the charts in one place

specs = [[{'type':'domain'}, {'type':'domain'}], [{'type':'domain'}, {'type':'domain'}]]
fig = make_subplots(rows=2, 
                    cols=2, 
                    specs=specs, 
                    subplot_titles=['Family History of Mental Illness <br><i>(% of respondents)</i>', 
                                    'Employed in a Tech Company <br><i>(% of respondents)</i>', 
                                    'Doing Remote Work <br><i>(% of respondents)</i>', 
                                    'Self Employed <br><i>(% of respondents)</i>'],
                    )
fig.update_annotations(font_size=12, yshift=10)
fig.add_trace(go.Pie(labels=df.family_history.value_counts().index,
                     values=df.family_history.value_counts().values,
                     name='',
                     hovertemplate = "# of respondents responding - %{label}: %{value}"), 1, 1)
fig.add_trace(go.Pie(labels=df.tech_company.value_counts().sort_values().index,
                     values=df.tech_company.value_counts().sort_values().values,
                     name='',
                     hovertemplate = "# of respondents responding - %{label}: %{value}"), 1, 2)
fig.add_trace(go.Pie(labels=df.remote_work.value_counts().index,
                     values=df.remote_work.value_counts().values,
                     name='',
                     hovertemplate = "# of respondents responding - %{label}: %{value}"), 2, 1)
fig.add_trace(go.Pie(labels=df.self_employed.value_counts().index,
                     values=df.self_employed.value_counts().values,
                     name='',
                     hovertemplate = "# of respondents responding - %{label}: %{value}"), 2, 2)
fig.update_traces(#hoverinfo='label+value', 
                  #textinfo='percent', 
                  textfont_size=12,
                  textposition='inside',
                  marker=dict(colors=['F9975D','B2EA70'],
                              line=dict(color='#000000', width=0.5)))
fig.update(layout_showlegend=True)

fig.add_annotation(x=0,
                   y=0,
            text="Respondent Profile (IV) <br>(n = 1244)",
            showarrow=False,
            xshift=450,
            yshift=370,            
            font={'size':14})
fig.add_annotation(x=0,
                   y=0,
            text="<i>Question text: 1. Do you have a family history of mental illness? (Yes / No)</i>",
            showarrow=False,
            xshift=0,
            yshift=-35,            
            font={'size':10})
fig.add_annotation(x=0,
                   y=0,
            text="<i>                     2. Is your employer primarily a tech company/organization? (Yes / No)</i>",
            showarrow=False,
            xshift=0,
            yshift=-47,            
            font={'size':10})
fig.add_annotation(x=0,
                   y=0,
            text="<i>                     3. Do you work remotely (outside of an office) at least 50% of the time? (Yes / No)</i>",
            showarrow=False,
            xshift=0,
            yshift=-59,            
            font={'size':10})
fig.add_annotation(x=0,
                   y=0,
            text="<i>                     4. Are you self-employed? (Yes / No)</i>",
            showarrow=False,
            xshift=0,
            yshift=-71,            
            font={'size':10})
fig.update_layout(title='',
                legend_y=0.5)
fig = go.Figure(fig)

fig.show()

* More than half the respondents (~60%) belong to families that do not have history of mental illness
* ~80% of the respondents are employed in a tech company 
* Only ~11% of the respondents reported being self-employed
* A little more than 2/3rd respondents are not engaged in remote work

### Main Section

#### Awareness of Mental Health
1. Among the employees who have not sought treatment for a mental health conditions ~80% belong to families with no history of mental illness. This could be due to a lack of awareness or exposure to resources.
2. More evidence for this is observed when we look at the awareness of care options among respondents who have and haven't sought treatment for a mental health condition. ~50% of treatment seekers report being aware of employer provided care options while ~50% of the non-treatment seekers report being unaware 
3. Overall only ~1/3rd of the respondents reported being aware of care options.

In [52]:
fig = go.Figure(data=[
    go.Bar(x=['Yes (sought treatment)','No (did not seek treatment)'], 
           y=df[df['family_history']=='Yes'].treatment.value_counts().values, 
           marker_color='#F05D5E', 
           name='Have a family history of mental illness',
           textfont=dict(color='#FFFFFF'),
           text=df[df['family_history']=='Yes'].treatment.value_counts().values,
           width=0.25,
           hoverinfo = 'skip'),
    go.Bar(x=['No (did not seek treatment)','Yes (sought treatment)'], 
           y=df[df['family_history']=='No'].treatment.value_counts().values, 
           marker_color='#0F7173', 
           name='Do not have a family history of mental illness',
           text=df[df['family_history']=='No'].treatment.value_counts().values,
           width=0.25,
           hoverinfo = 'skip')
    ])

fig.update_traces(marker=dict(line=dict(color='#000000', width=0.6)))
fig.update_layout(barmode='stack', 
                  title='Respondents with a family history of mental illness are more likely to seek treatment themselves as compared to those without <br>a family history of mental illness',
                  bargap=0.5, 
                  xaxis_tickangle=-0, 
                  plot_bgcolor="#FFFFFF", 
                  uniformtext_minsize=5, 
                  uniformtext_mode='hide',
                  #height=800,
                  title_y=0.92,
                  #margin=dict(t=170),
                  #legend_x=0.8, 
                  legend_y=0.5)
fig.add_annotation(
            x=0,
            y=0,
            text='Treatment for Mental Health Condition Cut by Family History of Mental Illness<br>(n = 1244)',
            showarrow=False,
            xshift=300,
            yshift=330,
            font={'size':14})
fig.add_annotation(x=0,
                   y=0,
            text="<i>Question text: Does your employer provide mental health benefits? (Yes / No / Don't know)</i>",
            showarrow=False,
            xshift=0,
            yshift=-60,            
            font={'size':10})  
   
fig.update_xaxes(showgrid=False, linecolor="gray", linewidth=0.5)
fig.update_yaxes(range=[0,800], showgrid=True, linecolor="gray", linewidth=0.5, title='# of respondents', titlefont=dict(size=12))

fig.show()

In [53]:
fig = go.Figure(data=go.Pie(values=df.care_options.value_counts().values, 
             labels=df.care_options.value_counts().index,name='',hovertemplate = "# of respondents responding - %{label}: %{value}"))

fig.update_traces(marker=dict(colors=['F9975D','B2EA70','FBD148'], line=dict(color='#000000', width=0.6)))

fig.add_annotation(
            text="% of respondents",
            showarrow=False,
            xshift=0,
            yshift=-150,
            font={'size':11})
fig.add_annotation(
            text='Care Options at the Workplace<br>(n = 1244)',
            showarrow=False,
            xshift=0,
            yshift=170,
            font={'size':14})
fig.add_annotation(
            text="<i>Question text: Do you know the options for mental health care your employer provides? (Yes / No / Don't know)</i>",
            showarrow=False,
            xshift=-280,
            yshift=-185,
            font={'size':10})
fig.update_layout(
                  height=500,
                  title_y=0.92,
                  margin=dict(t=170),
                  legend_x=0.8, 
                  legend_y=0.5, 
                  title_text='~35% of respondents report that they know the options for mental health care that their employers provide.')

fig.show()

In [54]:
# Trying to show all the charts in one place

specs = [[{'type':'domain'}, {'type':'domain'}]]
fig = make_subplots(rows=1, 
                    cols=2, 
                    specs=specs, 
                    subplot_titles=['Sought Treatment for a Mental Health Issue<br><i>(n = 629 | % of respondents)</i>', 
                                    'Have not Sought Treatment for a Mental Health Issue<br><i>(n = 615 | % of respondents)</i>', 
                                   ],
                    )
fig.update_annotations(font_size=12, yshift=10)
fig.add_trace(go.Pie(labels=df[df['treatment']=='Yes'].care_options.value_counts()[["Yes", "No", "Not sure"]].index,
                     values=df[df['treatment']=='Yes'].care_options.value_counts()[["Yes", "No", "Not sure"]].values,
                     name='',
                     hovertemplate = "# of respondents responding - %{label}: %{value}"), 1, 1)
fig.add_trace(go.Pie(labels=df[df['treatment']=='No'].care_options.value_counts()[["Yes", "No", "Not sure"]].index,
                     values=df[df['treatment']=='No'].care_options.value_counts()[["Yes", "No", "Not sure"]].values,
                     name='',
                     hovertemplate = "# of respondents responding - %{label}: %{value}"), 1, 2)

fig.update_traces(#hoverinfo='label+value', 
                  #textinfo='percent', 
                  textfont_size=12,
                  textposition='inside',
                  marker=dict(colors=['B2EA70','F9975D','FBD148'],
                              line=dict(color='#000000', width=0.5)))
fig.update(layout_showlegend=True)

fig.add_annotation(x=0,
                   y=0,
            text="Care Option Awareness - Treatment seekers vs others",
            showarrow=False,
            xshift=330,
            yshift=300,            
            font={'size':14})
fig.add_annotation(x=0,
                   y=0,
            text="<i>Question text: Do you know the options for mental health care your employer provides? (Yes / No / Not sure)</i>",
            showarrow=False,
            xshift=0,
            yshift=-70,            
            font={'size':10})

fig.update_layout(title='Almost 50% of the respondents who have sought treatment for a mental health issue are aware of the care options for mental <br>health provided by their employer; on the other side, among the repondents who have not sought treatment for a mental health <br>issue ~47% report not being aware of any employer provided care options for mental health',
                legend_y=0.5,
                height=500,
                title_y=0.92,
                margin=dict(t=170),
                )
fig = go.Figure(fig)

fig.show()

#### Access to Mental Health Resources
1. There is a lack of resources setup in the workplace for employees to seek help or learn more about mental health issues. Only ~20% of the respondents report that their employer provides resources to learn about mental health and avenues to seek help at the workplace.
2. Companies with lesser than 100 employees do not seem to provide any mental health benefits as only ~36% of the repondents from these smaller companies reported getting mental health benefits.
3. ~83% of the respondents from large companies (with >100 employees) report that their employer provides mental health benefits.
4. Large companies (with >100 employees) are more likely to offer access to mental health initiatives as part of their employee wellness programs and resources to seek help and learn more about mental health.

In [55]:
fig = go.Figure(data=[
    go.Bar(x=df[df['benefits']=='Yes'].no_employees.value_counts()[['1-5', '6-25', '26-100', '100-500','500-1000', 'More than 1000']].index, 
           y=df[df['benefits']=='Yes'].no_employees.value_counts()[['1-5', '6-25', '26-100', '100-500','500-1000', 'More than 1000']].values, 
           marker_color='#0F7173',
           #17BEBB 
           name='Benefits - Yes',
           text=df[df['benefits']=='Yes'].no_employees.value_counts()[['1-5', '6-25', '26-100', '100-500','500-1000', 'More than 1000']].values,
           width=0.5,
           hoverinfo = 'skip'),
    go.Bar(x=df[df['benefits']=='No'].no_employees.value_counts()[['1-5', '6-25', '26-100', '100-500','500-1000', 'More than 1000']].index, 
           y=df[df['benefits']=='No'].no_employees.value_counts()[['1-5', '6-25', '26-100', '100-500','500-1000', 'More than 1000']].values, 
           marker_color='#F05D5E',
           #EF3E36 
           name='Benefits - No',
           text=df[df['benefits']=='No'].no_employees.value_counts()[['1-5', '6-25', '26-100', '100-500','500-1000', 'More than 1000']].values,
           textfont=dict(color='#FFFFFF'),
           width=0.5,
           hoverinfo = 'skip')
    ])
fig.update_traces(marker=dict(line=dict(color='#000000', width=0.6)))
fig.update_layout(barmode='stack', 
                  title='Larger companies are more likely to provide mental health benefits.',
                  bargap=0.5, 
                  xaxis_tickangle=-0, 
                  plot_bgcolor="#FFFFFF", 
                  uniformtext_minsize=5, 
                  uniformtext_mode='hide',
                  #height=500,
                  title_y=0.92,
                  #margin=dict(t=170),
                  #legend_x=0.8, 
                  legend_y=0.5, 
                  )

fig.add_annotation(
            text='Benefits Offered by Company Size<br>(n = 1244)',
            showarrow=False,
            xshift=0,
            yshift=170,
            font={'size':14})
fig.add_annotation(
            text="<i>Question text: Does your employer provide mental health benefits? (Yes / No / Don't know)</i>",
            showarrow=False,
            xshift=-280,
            yshift=-230,
            font={'size':10})

fig.update_xaxes(showgrid=False, linecolor="gray", linewidth=0.5)
fig.update_yaxes(range=[0,300], showgrid=True, linecolor="gray", linewidth=0.5, title='# of respondents', titlefont=dict(size=12))

fig.show()

In [56]:
fig = go.Figure(data=go.Pie(values=df.wellness_program.value_counts().values, 
             labels=df.wellness_program.value_counts().index,name='',hovertemplate = "# of respondents responding - %{label}: %{value}"))

fig.update_traces(marker=dict(colors=['F9975D','B2EA70','FBD148'], line=dict(color='#000000', width=0.6)))

fig.add_annotation(
            text="% of respondents",
            showarrow=False,
            xshift=0,
            yshift=-150,
            font={'size':11})
fig.add_annotation(
            text='The Employee Wellness Program Dicussion by Employer<br>(n = 1244)',
            showarrow=False,
            xshift=0,
            yshift=170,
            font={'size':14})
fig.add_annotation(
            text="<i>Question text: Has your employer ever discussed mental health as part of an employee wellness program? (Yes / No / Don't know)</i>",
            showarrow=False,
            xshift=-220,
            yshift=-185,
            font={'size':10})
fig.update_layout(
                  height=500,
                  title_y=0.92,
                  margin=dict(t=170),
                  legend_x=0.8, 
                  legend_y=0.5, 
                  title_text='An overwhelming majority (67%) of the respondents report that their employers did not have a discussion on their employee <br>wellness program.')

fig.show()

In [57]:
# Trying to show all the charts in one place

specs = [[{'type':'domain'}, {'type':'domain'}]]
fig = make_subplots(rows=1, 
                    cols=2, 
                    specs=specs, 
                    subplot_titles=['Small Scale Companies (<100 employees) <br><i>(n = 729 | % of respondents)</i>', 
                                    'Large Companies (>100 employees) <br><i>(n = 515 | % of respondents)</i>', 
                                   ],
                    )
fig.update_annotations(font_size=12, yshift=10)
fig.add_trace(go.Pie(labels=df[(df['no_employees']=='1-5') | (df['no_employees']=='6-25') | (df['no_employees']=='26-100')].wellness_program.value_counts()[["Yes", "No", "Don't know"]].index,
                     values=df[(df['no_employees']=='1-5') | (df['no_employees']=='6-25') | (df['no_employees']=='26-100')].wellness_program.value_counts()[["Yes", "No", "Don't know"]].values,
                     name='',
                     hovertemplate = "# of respondents responding - %{label}: %{value}"), 1, 1)
fig.add_trace(go.Pie(labels=df[(df['no_employees']=='100-500') | (df['no_employees']=='500-1000') | (df['no_employees']=='More than 1000')].wellness_program.value_counts()[["Yes", "No", "Don't know"]].index,
                     values=df[(df['no_employees']=='100-500') | (df['no_employees']=='500-1000') | (df['no_employees']=='More than 1000')].wellness_program.value_counts()[["Yes", "No", "Don't know"]].values,
                     name='',
                     hovertemplate = "# of respondents responding - %{label}: %{value}"), 1, 2)

fig.update_traces(#hoverinfo='label+value', 
                  #textinfo='percent', 
                  textfont_size=12,
                  textposition='inside',
                  marker=dict(colors=['B2EA70','F9975D','FBD148'],
                              line=dict(color='#000000', width=0.5)))
fig.update(layout_showlegend=True)

fig.add_annotation(x=0,
                   y=0,
            text="Mental Health as part of Wellness Programs<br>Small Scale vs Large Companies",
            showarrow=False,
            xshift=370,
            yshift=300,            
            font={'size':14})
fig.add_annotation(x=0,
                   y=0,
            text="<i>Question text: Has your employer ever discussed mental health as part of an employee wellness program? (Yes / No / Don't know)</i>",
            showarrow=False,
            xshift=0,
            yshift=-70,            
            font={'size':10})

fig.update_layout(title='Employees from large companies (>100 employees) are more likely to have mental health initiatives as part of their wellness <br>programs.',
                legend_y=0.5,
                height=500,
                title_y=0.92,
                margin=dict(t=170),
                )
fig = go.Figure(fig)

fig.show()

In [23]:
fig = go.Figure(data=go.Pie(values=df.seek_help.value_counts().values, 
             labels=df.seek_help.value_counts().index,name='',hovertemplate = "# of respondents responding - %{label}: %{value}"))

fig.update_traces(marker=dict(colors=['F9975D','FBD148','B2EA70'], line=dict(color='#000000', width=0.6)))

fig.add_annotation(
            text="% of respondents",
            showarrow=False,
            xshift=0,
            yshift=-150,
            font={'size':11})
fig.add_annotation(
            text='Oppurtunities and Resources to Seek Help at the Workplace<br>(n = 1244)',
            showarrow=False,
            xshift=0,
            yshift=170,
            font={'size':14})
fig.add_annotation(
            text="<i>Question text: Does your employer provide resources to learn more about mental health issues and how to seek help? (Yes / No / Don't know)</i>",
            showarrow=False,
            xshift=-220,
            yshift=-185,
            font={'size':10})
fig.update_layout(
                  height=500,
                  title_y=0.92,
                  margin=dict(t=170),
                  legend_x=0.8, 
                  legend_y=0.5, 
                  title_text='Almost half of the respondents (~51%) report that their employers do not provide resources to learn about mental health or <br>any information on how to seek help.')

fig.show()

In [24]:
# Trying to show all the charts in one place

specs = [[{'type':'domain'}, {'type':'domain'}]]
fig = make_subplots(rows=1, 
                    cols=2, 
                    specs=specs, 
                    subplot_titles=['Small Scale Companies (<100 employees) <br><i>(n = 729 | % of respondents)</i>', 
                                    'Large Companies (>100 employees) <br><i>(n = 515 | % of respondents)</i>', 
                                   ],
                    )
fig.update_annotations(font_size=12, yshift=10)
fig.add_trace(go.Pie(labels=df[(df['no_employees']=='1-5') | (df['no_employees']=='6-25') | (df['no_employees']=='26-100')].seek_help.value_counts()[["Yes", "No", "Don't know"]].index,
                     values=df[(df['no_employees']=='1-5') | (df['no_employees']=='6-25') | (df['no_employees']=='26-100')].seek_help.value_counts()[["Yes", "No", "Don't know"]].values,
                     name='',
                     hovertemplate = "# of respondents responding - %{label}: %{value}"), 1, 1)
fig.add_trace(go.Pie(labels=df[(df['no_employees']=='100-500') | (df['no_employees']=='500-1000') | (df['no_employees']=='More than 1000')].seek_help.value_counts()[["Yes", "No", "Don't know"]].index,
                     values=df[(df['no_employees']=='100-500') | (df['no_employees']=='500-1000') | (df['no_employees']=='More than 1000')].seek_help.value_counts()[["Yes", "No", "Don't know"]].values,
                     name='',
                     hovertemplate = "# of respondents responding - %{label}: %{value}"), 1, 2)

fig.update_traces(#hoverinfo='label+value', 
                  #textinfo='percent', 
                  textfont_size=12,
                  textposition='inside',
                  marker=dict(colors=['B2EA70','F9975D','FBD148'],
                              line=dict(color='#000000', width=0.5)))
fig.update(layout_showlegend=True)

fig.add_annotation(x=0,
                   y=0,
            text="Oppurtunities and Resources to Seek Help in the Workplace<br>Small Scale vs Large Companies",
            showarrow=False,
            xshift=310,
            yshift=300,            
            font={'size':14})
fig.add_annotation(x=0,
                   y=0,
            text="<i>Question text: Does your employer provide resources to learn more about mental health issues and how to seek help? (Yes / No / Don't know)</i>",
            showarrow=False,
            xshift=0,
            yshift=-70,            
            font={'size':10})

fig.update_layout(title='Employees from large companies (>100 employees) are more likely to have access to oppurtunities and resources to seek help <br>and learn more about mental health in their workplace.',
                legend_y=0.5,
                height=500,
                title_y=0.92,
                margin=dict(t=170),
                )
fig = go.Figure(fig)

fig.show()

#### Ease of Managing Mental Health for an Employee (as faciliated by the employer)
1. Taking leaves for a mental health condition is still a challenge as only 37% of respondents reported that they found it easy to take leaves for a mental health condition.
2. Protection of anonymity for those who choose to take advantage of resources for mental health seems to be uncertain as 65% of the respondents report not knowing of any such provisions in their workplace.

In [25]:
pd.options.display.float_format = '${:,.2f}'.format

fig_value = ((df.leave.value_counts()[["Don't know", "Very easy", "Somewhat easy", "Somewhat difficult", "Very difficult"]].values)*100/sum(df.leave.value_counts()[["Don't know", "Very easy", "Somewhat easy", "Somewhat difficult", "Very difficult"]].values)).astype(int)
a = fig_value.astype(str).tolist()

b = ['%']*5
final_value = [y + z for (y, z) in zip(a, [str(i) for i in b])]
c = df.leave.value_counts()[["Don't know", "Very easy", "Somewhat easy", "Somewhat difficult", "Very difficult"]].values.astype(str).tolist()
d = final_value
final_text = [z + ' ' + '(' + y + ')' for (y, z) in zip(c, [str(i) for i in d])]
fig = go.Figure(go.Bar(x=df.leave.value_counts()[["Don't know", "Very easy", "Somewhat easy", "Somewhat difficult", "Very difficult"]].index, 
                       y=df.leave.value_counts()[["Don't know", "Very easy", "Somewhat easy", "Somewhat difficult", "Very difficult"]].values/sum(df.leave.value_counts()[["Don't know", "Very easy", "Somewhat easy", "Somewhat difficult", "Very difficult"]].values),
                       width=0.4,
                       name='',
                       text=final_text,                    
                       hoverinfo='skip'))

fig.update_traces(marker=dict(color='#D4355A', line=dict(color='#000000', width=0.6)))

# fig.add_annotation(
#             text="% of respondents",
#             showarrow=False,
#             xshift=-300,
#             yshift=-200,
#             font={'size':11})
fig.add_annotation(
            text='Ease of Obtaining Medical Leave due to Mental Health<br>(n = 1244)',
            showarrow=False,
            xshift=-120,
            yshift=170,
            font={'size':14})
fig.add_annotation(
            text="<i>Question text: How easy is it for you to take medical leave for a mental health condition? (Very easy / Somewhat easy / Don't know / Somewhat difficult / Very difficult)</i>",
            showarrow=False,
            xshift=-220,
            yshift=-230,
            font={'size':10})

fig.update_xaxes(showgrid=False, linecolor="gray", linewidth=0.5)
fig.update_yaxes(tickformat= ',.0%', range=[0,1], showgrid=True, linecolor="gray", linewidth=0.5, title='% of respondents', titlefont=dict(size=12, family='verdana'))

fig.update_layout(plot_bgcolor="#FFFFFF", 
                  #barmode='stack',
                  #height=500,
                  title_y=0.92,
                  bargap=0.2,
                  #margin=dict(t=170),
                  #legend_x=0.8, 
                  #legend_y=0.5, 
                  title_text='While only 16% of the respondents reported difficulty obtaining medical leaves due to their mental health conditions, 44% of <br>the respondents were unable to effectively classify their difficulties into the given buckets suggesting further nuances present <br>in the issue.')

fig.show()

In [26]:
fig = go.Figure(data=go.Pie(values=df.anonymity.value_counts().values, 
             labels=df.anonymity.value_counts().index,name='',hovertemplate = "# of respondents responding - %{label}: %{value}"))

fig.update_traces(marker=dict(colors=['FBD148','B2EA70','F9975D'], line=dict(color='#000000', width=0.6)))

fig.add_annotation(
            text="% of respondents",
            showarrow=False,
            xshift=0,
            yshift=-150,
            font={'size':11})
fig.add_annotation(
            text='Anonymity When Utilising Mental Health Resources<br>(n = 1244)',
            showarrow=False,
            xshift=0,
            yshift=170,
            font={'size':14})
fig.add_annotation(
            text="<i>Question text: Is your anonymity protected if you choose to take advantage of mental health or substance abuse treatment resources? (Yes / No / Don't know)</i>",
            showarrow=False,
            xshift=-160,
            yshift=-185,
            font={'size':10})
fig.update_layout(
                  height=500,
                  title_y=0.92,
                  margin=dict(t=170),
                  legend_x=0.8, 
                  legend_y=0.5, 
                  title_text='There is a lack of awareness on whether anonymity is protected if employees choose to utilise mental health or substance <br>abuse treatment resources as 65% of respondents reported not knowing of such provisions.')

fig.show()

####Consequences of Discussing mental and physical health issues with Employers
#####1. Respondents find it easier to discuss their physical health issues with their employers as compared to their mental health issues. 
#####2. More than 2/3rd (74%) of the respondents believe that there would be no negative consequence to discussing a physical health issue; while only ~40% believe that there would be no consequence to discussing a mental health issue.

In [27]:
fig = go.Figure(data=go.Pie(values=df.mental_health_consequence.value_counts().values, 
             labels=df.mental_health_consequence.value_counts().index,name='',hovertemplate = "# of respondents responding - %{label}: %{value}"))

fig.update_traces(marker=dict(colors=['F9975D','FBD148','B2EA70'], line=dict(color='#000000', width=0.6)))

fig.add_annotation(
            text="% of respondents",
            showarrow=False,
            xshift=0,
            yshift=-150,
            font={'size':11})
fig.add_annotation(
            text='Mental Health Consequence<br>(n = 1244)',
            showarrow=False,
            xshift=0,
            yshift=170,
            font={'size':14})
fig.add_annotation(
            text="<i>Question text: Do you think that discussing a mental health issue with your employer would have negative consequences? (Yes / No / Maybe)</i>",
            showarrow=False,
            xshift=-180,
            yshift=-185,
            font={'size':10})
fig.update_layout(
                  height=500,
                  title_y=0.92,
                  margin=dict(t=170),
                  legend_x=0.8, 
                  legend_y=0.5, 
                  title_text='Only ~40% of the respondents report that they think there would no negative consequences to dicussing a mental health issue <br>with their employers.')

fig.show()

In [28]:
fig = go.Figure(data=go.Pie(values=df.phys_health_consequence.value_counts().values, 
             labels=df.phys_health_consequence.value_counts().index,name='',hovertemplate = "# of respondents responding - %{label}: %{value}"))

fig.update_traces(marker=dict(colors=['F9975D','FBD148','B2EA70'], line=dict(color='#000000', width=0.6)))

fig.add_annotation(
            text="% of respondents",
            showarrow=False,
            xshift=0,
            yshift=-150,
            font={'size':11})
fig.add_annotation(
            text='Physical Health Consequence<br>(n = 1244)',
            showarrow=False,
            xshift=0,
            yshift=170,
            font={'size':14})
fig.add_annotation(
            text="<i>Question text: Do you think that discussing a physical health issue with your employer would have negative consequences? (Yes / No / Maybe)</i>",
            showarrow=False,
            xshift=-180,
            yshift=-185,
            font={'size':10})
fig.update_layout(
                  height=500,
                  title_y=0.92,
                  margin=dict(t=170),
                  legend_x=0.8, 
                  legend_y=0.5, 
                  title_text='~74% of the respondents report that they think there would no negative consequences to dicussing a physical health issue <br>with their employers.')

fig.show()

####Discussing Mental Health: Coworkers and Supervisors
#####1. Respondents seem to be more inclined to discuss their mental health issues with their supervisors as compared to their coworkers.
#####2. 18% of respondents were open to discussing their mental health issues with their coworkers while 40% who were open to such a discussion with their supervisors

In [29]:
fig = go.Figure(data=go.Pie(values=df.coworkers.value_counts().values, 
             labels=df.coworkers.value_counts().index,name='',hovertemplate = "# of respondents responding - %{label}: %{value}"))

fig.update_traces(marker=dict(colors=['FBD148','F9975D','B2EA70'], line=dict(color='#000000', width=0.6)))

fig.add_annotation(
            text="% of respondents",
            showarrow=False,
            xshift=0,
            yshift=-150,
            font={'size':11})
fig.add_annotation(
            text='Coworkers: Mental Health<br>(n = 1244)',
            showarrow=False,
            xshift=0,
            yshift=170,
            font={'size':14})
fig.add_annotation(
            text="<i>Question text: Would you be willing to discuss a mental health issue with your coworkers? (Yes / No / Some of them)</i>",
            showarrow=False,
            xshift=-250,
            yshift=-185,
            font={'size':10})
fig.update_layout(
                  height=500,
                  title_y=0.92,
                  margin=dict(t=170),
                  legend_x=0.8, 
                  legend_y=0.5, 
                  title_text='~83% of the respondents report that they would not or are unsure if they would be willing to dicuss their mental health issues <br>with their coworkers.')

fig.show()

In [30]:
fig = go.Figure(data=go.Pie(values=df.supervisor.value_counts().values, 
             labels=df.supervisor.value_counts().index,name='',hovertemplate = "# of respondents responding - %{label}: %{value}"))

fig.update_traces(marker=dict(colors=['B2EA70','F9975D','FBD148'], line=dict(color='#000000', width=0.6)))

fig.add_annotation(
            text="% of respondents",
            showarrow=False,
            xshift=0,
            yshift=-150,
            font={'size':11})
fig.add_annotation(
            text='Supervisor: Mental Health<br>(n = 1244)',
            showarrow=False,
            xshift=0,
            yshift=170,
            font={'size':14})
fig.add_annotation(
            text="<i>Question text: Would you be willing to discuss a mental health issue with your direct supervisor(s)? (Yes / No / Some of them)</i>",
            showarrow=False,
            xshift=-250,
            yshift=-185,
            font={'size':10})
fig.update_layout(
                  height=500,
                  title_y=0.92,
                  margin=dict(t=170),
                  legend_x=0.8, 
                  legend_y=0.5, 
                  title_text='~60% of the respondents report that they would not or are unsure if they would bring up a mental health issue to a supervisor.')

fig.show()

#### Impact of Mental Health on Recruitment Interviews
1. Almost 80% of the respondents reported that they would not bring up a mental health condition with a potential employer during an interview.
2. Physical health conditions face lesser stigma as a comparitively low 40% of repondents say that they would not talk about any physical health conditions during an interview.

In [31]:
fig = go.Figure(data=go.Pie(values=df.mental_health_interview.value_counts().values, 
             labels=df.mental_health_interview.value_counts().index,name='',hovertemplate = "# of respondents responding - %{label}: %{value}"))

fig.update_traces(marker=dict(colors=['F9975D','FBD148','B2EA70'], line=dict(color='#000000', width=0.6)))

fig.add_annotation(
            text="% of respondents",
            showarrow=False,
            xshift=0,
            yshift=-150,
            font={'size':11})
fig.add_annotation(
            text='Interview: Mental Health<br>(n = 1244)',
            showarrow=False,
            xshift=0,
            yshift=170,
            font={'size':14})
fig.add_annotation(
            text="<i>Question text: Would you bring up a mental health issue with a potential employer in an interview? (Yes / No / Maybe)</i>",
            showarrow=False,
            xshift=-250,
            yshift=-185,
            font={'size':10})
fig.update_layout(
                  height=500,
                  title_y=0.92,
                  margin=dict(t=170),
                  legend_x=0.8, 
                  legend_y=0.5, 
                  title_text='~97% of the respondents report that they would not or are unsure if they would bring up a mental health issue during an interview.')

fig.show()

In [32]:
fig = go.Figure(data=go.Pie(values=df.phys_health_interview.value_counts().values, 
             labels=df.phys_health_interview.value_counts().index,name='',hovertemplate = "# of respondents responding - %{label}: %{value}"))

fig.update_traces(marker=dict(colors=['FBD148','F9975D','B2EA70'], line=dict(color='#000000', width=0.6)))

fig.add_annotation(
            text="% of respondents",
            showarrow=False,
            xshift=0,
            yshift=-150,
            font={'size':11})
fig.add_annotation(
            text='Interview: Phyical Health<br>(n = 1244)',
            showarrow=False,
            xshift=0,
            yshift=170,
            font={'size':14})
fig.add_annotation(
            text="<i>Question text: Would you bring up a physical health issue with a potential employer in an interview? (Yes / No / Maybe)</i>",
            showarrow=False,
            xshift=-250,
            yshift=-185,
            font={'size':10})
fig.update_layout(
                  height=500,
                  title_y=0.92,
                  margin=dict(t=170),
                  legend_x=0.8, 
                  legend_y=0.5, 
                  title_text='84% of the respondents report that they would not or are unsure if they would bring up a physical health issue during an interview.')

fig.show()

#### Impact of Mental Health on Performance at Work
1. Almost half of the respondents report that their mental heath condition interferes with their work.
2. Only ~27% of the respondents feel that their employers take mental health as seriously as physical health, however despite this low number, only ~14% of the respondents report hearing or observing their coworkers face negative consequences due to a mental health condition.

In [33]:
pd.options.display.float_format = '${:,.2f}'.format

fig_value = ((df.work_interfere.value_counts()[['Never', 'Rarely', 'Sometimes', 'Often']].values)*100/sum(df.work_interfere.value_counts()[['Never', 'Rarely', 'Sometimes', 'Often']].values)).astype(int)
a = fig_value.astype(str).tolist()

b = ['%']*5
final_value = [y + z for (y, z) in zip(a, [str(i) for i in b])]
c = df.work_interfere.value_counts()[['Never', 'Rarely', 'Sometimes', 'Often']].values.astype(str).tolist()
d = final_value
final_text = [z + ' ' + '(' + y + ')' for (y, z) in zip(c, [str(i) for i in d])]
fig = go.Figure(go.Bar(x=df.work_interfere.value_counts()[['Never', 'Rarely', 'Sometimes', 'Often']].index, 
                       y=df.work_interfere.value_counts()[['Never', 'Rarely', 'Sometimes', 'Often']].values/sum(df.work_interfere.value_counts()[['Never', 'Rarely', 'Sometimes', 'Often']].values),
                       width=0.4,
                       name='',
                       text=final_text,                    
                       hoverinfo='skip'))

fig.update_traces(marker=dict(color='#D4355A', line=dict(color='#000000', width=0.6)))

# fig.add_annotation(
#             text="% of respondents",
#             showarrow=False,
#             xshift=-300,
#             yshift=-200,
#             font={'size':11})
fig.add_annotation(
            x=0,
            y=0,
            text='Work Interference due to Mental Health<br>(n = 984)',
            showarrow=False,
            xshift=370,
            yshift=350,
            font={'size':14})
fig.add_annotation(
            text="<i>Question text: If you have a mental health condition, do you feel that it interferes with your work? (Often / Sometimes / Rarely / Never)</i>",
            showarrow=False,
            xshift=-450,
            yshift=-230,
            font={'size':10})

fig.update_xaxes(showgrid=False, linecolor="gray", linewidth=0.5)
fig.update_yaxes(tickformat= ',.0%', range=[0,1], showgrid=True, linecolor="gray", linewidth=0.5, title='% of respondents', titlefont=dict(size=12, family='verdana'))

fig.update_layout(plot_bgcolor="#FFFFFF", 
                  #barmode='stack',
                  #height=500,
                  title_y=0.92,
                  bargap=0.2,
                  #margin=dict(t=170),
                  #legend_x=0.8, 
                  #legend_y=0.5, 
                  title_text='Only ~21% of the respondents report that their mental health conditions have not interfered with their work.')

fig.show()

In [34]:
fig = go.Figure(data=go.Pie(values=df.mental_vs_physical.value_counts().values, 
             labels=df.mental_vs_physical.value_counts().index,name='',hovertemplate = "# of respondents responding - %{label}: %{value}"))

fig.update_traces(marker=dict(colors=['FBD148','B2EA70','F9975D'], line=dict(color='#000000', width=0.6)))

fig.add_annotation(
            text="% of respondents",
            showarrow=False,
            xshift=0,
            yshift=-150,
            font={'size':11})
fig.add_annotation(
            text='Employer consideration: Mental Health vs Phyical Health<br>(n = 1244)',
            showarrow=False,
            xshift=0,
            yshift=170,
            font={'size':14})
fig.add_annotation(
            text="<i>Question text: Do you feel that your employer takes mental health as seriously as physical health? (Yes / Don't know / No)</i>",
            showarrow=False,
            xshift=-250,
            yshift=-185,
            font={'size':10})
fig.update_layout(
                  height=500,
                  title_y=0.92,
                  margin=dict(t=170),
                  legend_x=0.8, 
                  legend_y=0.5, 
                  title_text='Only ~27% of the respondents report that their employers treat mental health with the same seriousness as physical health.')

fig.show()

In [35]:
fig = go.Figure(data=go.Pie(values=df.obs_consequence.value_counts().values, 
             labels=df.obs_consequence.value_counts().index,name='',hovertemplate = "# of respondents responding - %{label}: %{value}"))

fig.update_traces(marker=dict(colors=['F9975D','B2EA70'], line=dict(color='#000000', width=0.6)))

fig.add_annotation(
            text="% of respondents",
            showarrow=False,
            xshift=0,
            yshift=-150,
            font={'size':11})
fig.add_annotation(
            text='Observed Negative Consequences for Coworkers with Mental Health Conditions<br>(n = 1244)',
            showarrow=False,
            xshift=0,
            yshift=170,
            font={'size':14})
fig.add_annotation(
            text="<i>Question text: Have you heard of or observed negative consequences for coworkers with mental health conditions in your workplace? (Yes / No)</i>",
            showarrow=False,
            xshift=-200,
            yshift=-185,
            font={'size':10})
fig.update_layout(
                  height=500,
                  title_y=0.92,
                  margin=dict(t=170),
                  legend_x=0.8, 
                  legend_y=0.5, 
                  title_text='Only ~14% of the respondents report that they observed negative consequences for coworkers with mental health conditions.')

fig.show()

###Reference Section

#####Questions in the survey


1. Timestamp

2. Age

3. Gender

4. Country

5. state: If you live in the United States, which state or territory do you live in?

6. self_employed: Are you self-employed? (Yes / No)

7. family_history: Do you have a family history of mental illness?  (Yes / No)

8. treatment: Have you sought treatment for a mental health condition? (Yes / No)

9. work_interfere: If you have a mental health condition, do you feel that it interferes with your work? (Often / Rarely / Never / Sometimes)

10. no_employees: How many employees does your company or organization have? 
(1-5 / 6-25 / 26-100 / 100-500 / 500-1000 / More than 1000 )

11. remote_work: Do you work remotely (outside of an office) at least 50% of the time? (Yes / No)

12. tech_company: Is your employer primarily a tech company/organization? (Yes / No)

13. benefits: Does your employer provide mental health benefits?
(Yes / Don't know / No)

14. care_options: Do you know the options for mental health care your employer provides? (Yes / No / Not sure)

15. wellness_program: Has your employer ever discussed mental health as part of an employee wellness program? (Yes / Don't know / No)

16. seek_help: Does your employer provide resources to learn more about mental health issues and how to seek help? (Yes / Don't know / No)

17. anonymity: Is your anonymity protected if you choose to take advantage of mental health or substance abuse treatment resources? (Yes / Don't know / No)

18. leave: How easy is it for you to take medical leave for a mental health condition? (Very easy/ Somewhat easy / Don't know / Somewhat difficult / Very difficult)

19. mentalhealthconsequence: Do you think that discussing a mental health issue with your employer would have negative consequences? (Yes / No / Maybe)

20. physhealthconsequence: Do you think that discussing a physical health issue with your employer would have negative consequences? (Yes / No / Maybe)

21. coworkers: Would you be willing to discuss a mental health issue with your coworkers? (Yes / No / Some of them)

22. supervisor: Would you be willing to discuss a mental health issue with your direct supervisor(s)? (Yes / No / Some of them)

23. mentalhealthinterview: Would you bring up a mental health issue with a potential employer in an interview? (Yes / No / Maybe)

24. physhealthinterview: Would you bring up a physical health issue with a potential employer in an interview? (Yes / No / Maybe)

25. mentalvsphysical: Do you feel that your employer takes mental health as seriously as physical health? (Yes / Don't know / No)

26. obs_consequence: Have you heard of or observed negative consequences for coworkers with mental health conditions in your workplace? (Yes / No)

27. comments: Any additional notes or comments